In [ ]:
### Importando bibliotecas necessárias

In [ ]:
!pip install matplotlib --index-url https://pypi.org/simple/
!pip install scikit-learn --index-url https://pypi.org/simple/
!pip install scikit-learn scikit-image --index-url https://pypi.org/simple

In [ ]:
#from dadosfera import *
import snowflake.connector
import pandas as pd
import numpy as np
import os
#from joblib import dump
#import matplotlib.pyplot as plt
from operator import itemgetter
#import seaborn as sns
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import orchest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

### Acessando os dados armazenados com 'output'

In [ ]:
data = orchest.get_inputs() 
print(data.keys())
all_tables=data['all_tables']
clients = all_tables['clients']
cities = all_tables['cities']
states = all_tables['states']
products = all_tables['products']
carts = all_tables['carts']
transactions = all_tables['transactions']
deliveries = all_tables['deliveries']
behaviors = all_tables['behaviors']
# clients = data['dict_df']['clients']
# cities = data['dict_df']['cities']
# states = data['dict_df']['states']
# products = data['dict_df']['products']
# carts = data['dict_df']['carts']
# transactions = data['dict_df']['transactions']
# deliveries = data['dict_df']['deliveries']
# behaviors = data['dict_df']['behaviors']
transactions.head()

In [ ]:
kpis={}
### Calculando o ticket médio
total_sum = transactions['TotalValue'].astype(float).sum()  # Somando todos os valores da coluna TotalSum
num_transactions = transactions.shape[0]  # Contando o número de transações (linhas)
ticket_medio = total_sum / num_transactions if num_transactions > 0 else 0  # Evita divisão por zero
print("\nTicket médio:")
print(ticket_medio)
kpis['ticket_medio'] = ticket_medio
### Calculando o valor e o número de transações por período
# Criar uma nova coluna 'year' e 'month_year'
transactions['year'] = transactions['Date'].dt.year
transactions['month_year'] = transactions['Date'].dt.to_period('M')

# Agrupar por 'year' e calcular a soma de TotalSum e o número de transações
transactions_yearly = transactions.groupby('year').agg(
    total_sales_year=('TotalValue', 'sum'),        # Soma dos valores de TotalSum
    total_transactions_year=('TotalValue', 'size') # Contagem de transações
).reset_index()

# Agrupar por 'month_year' e calcular a soma de TotalSum e o número de transações
transactions_monthly = transactions.groupby('month_year').agg(
    total_sales_month=('TotalValue', 'sum'),       # Soma dos valores de TotalSum
    total_transactions_month=('TotalValue', 'size')# Contagem de transações
).reset_index()

# Exibir os resultados
print("Total de vendas e número de transações por ano:")
print(transactions_yearly)
kpis['transactions_yearly'] = transactions_yearly

print("\nTotal de vendas e número de transações por mês-ano:")
print(transactions_monthly)
kpis['transactions_monthly'] = transactions_monthly

### Lucro Líquido
# Considerando que o lucro líquido hipotético é uma margem de 30% o valor do produto.
# A coluna 'ProductId' é a chave de ligação entre transactions e products.
# Fazendo a junção (merge) entre as tabelas transactions e products usando a coluna 'ProductId'
transactions_products = transactions.merge(products[['ProductId', 'Price']], on='ProductId', how='left')

# Calculando o preço de custo (70% do preço de venda)
transactions_products['PriceCost'] = transactions_products['Price'] * 0.7

# Calculando o lucro: Lucro = TotalSum - PrecoCusto
# Se a coluna 'TotalSum' representa o valor pago na transação
transactions_products['Profit'] = transactions_products['TotalValue'] - transactions_products['PriceCost']

# Exibir as primeiras linhas para verificar
print(transactions_products[['ProductId', 'TotalValue', 'Price', 'PriceCost', 'Profit']].head())

# Calcula o Lucro Líquido Total somando todos os valores da coluna 'Lucro'
profit = transactions_products['Profit'].sum()
invoicing = transactions_products['TotalValue'].sum()

print("\nFaturamento total:")
kpis['invoicing'] = invoicing
print(invoicing)

print("\nLucro Líquido Total:")
kpis['profit'] = profit
print(profit)

### Crescimento da Receita

# Certifique-se de que os dados estão ordenados por ano
transactions_yearly = transactions_yearly.sort_values('year')

# Calcular o crescimento da receita de um ano para o outro
transactions_yearly['growth_percentage'] = transactions_yearly['total_sales_year'].pct_change() * 100

# Exibir os resultados
print("Crescimento da receita por ano:")
print(transactions_yearly)
kpis['growth_percentage_yearly'] = transactions_yearly

### Taxa de conversão
# Taxa de Conversão = Número de Visitantes / Número de Compras × 100
# Remover duplicatas para garantir contagem precisa de visitantes únicos
unique_visitors = behaviors['CustomerId'].drop_duplicates().count()

# Identificar clientes que realizaram pelo menos uma compra
purchasing_customers = transactions['CustomerId'].drop_duplicates()

# Número de clientes que realizaram compras
num_purchasing_customers = purchasing_customers.count()

# Calcular a taxa de conversão
conversion_rate = (num_purchasing_customers / unique_visitors) * 100

# Exibir o resultado
print(f"Taxa de Conversão: {conversion_rate:.2f}%")
kpis['conversion_rate'] = conversion_rate

### Maiores LTV de clientes
# Calcular o LTV para cada cliente
ltv_per_customer = transactions.groupby('CustomerId')['TotalValue'].sum().reset_index()

# Criar uma nova coluna 'LTV' no DataFrame original
transactions = transactions.merge(ltv_per_customer, on='CustomerId', suffixes=('', '_LTV'))

# Ordenar os clientes pelo LTV em ordem decrescente
ltv_per_customer_sorted = ltv_per_customer.sort_values(by='TotalValue', ascending=False)

# Selecionar os 10 melhores clientes com maior LTV
top_10_customers_ltv = ltv_per_customer_sorted.head(10)

# Exibir os resultados
print("LTV dos 10 melhores clientes:")
print(top_10_customers_ltv)
kpis['top_10_customers_ltv'] = top_10_customers_ltv

# Exibir os dados com a nova coluna 'LTV'
print("\nDataFrame com a nova coluna 'LTV':")
print(transactions.head())

### Número de clientes ativos nos últimos 30 dias que realizaram compras
end_date = datetime.now()
#start_date = end_date - pd.DateOffset(months=6) 
start_date = end_date - timedelta(days=30)

# Filtrar transações dentro do período de tempo definido
active_transactions = transactions[(transactions['Date'] >= start_date) & (transactions['Date'] <= end_date)]

# Identificar clientes únicos que realizaram transações no período
active_customers = active_transactions['CustomerId'].drop_duplicates()

# Número de clientes ativos
num_active_customers_30_days = active_customers.count()

# Exibir o resultado
print(f"Número de clientes ativos nos últimos  dias: {num_active_customers_30_days}")
kpis['num_active_customers_30_days'] = num_active_customers_30_days

### Taxa de retenção
# Percentual de clientes que retornam e compram novamente em um determinado período. Indica a fidelidade dos clientes e a eficácia das estratégias de retenção.
# Agrupar clientes únicos por mês-ano
customers_per_month = active_transactions.groupby('month_year')['CustomerId'].apply(lambda x: x.nunique()).reset_index(name='unique_customers')

# Criar uma coluna para a quantidade de clientes retornados de um mês para outro
customers_per_month['retained_customers'] = customers_per_month['unique_customers'].shift(-1)

# Calcular a taxa de retenção
customers_per_month['retention_rate'] = (customers_per_month['retained_customers'] / customers_per_month['unique_customers']) * 100

# Exibir os resultados
print(customers_per_month[['month_year', 'unique_customers', 'retained_customers', 'retention_rate']])
kpis['retention_rate'] = customers_per_month[['month_year', 'retention_rate']]

### Taxa de Aquisição de clientes mensal
# Número de novos clientes adquiridos em um período de tempo específico.

# Identificar a primeira compra de cada cliente
first_purchase = transactions.groupby('CustomerId')['Date'].min().reset_index()
first_purchase['month_year'] = first_purchase['Date'].dt.to_period('M')

# Contar o número de novos clientes adquiridos por mês
new_customers_per_month = first_purchase.groupby('month_year')['CustomerId'].nunique().reset_index(name='new_customers')

# Contar o total de clientes únicos por mês (não apenas novos)
total_customers_per_month = transactions.groupby('month_year')['CustomerId'].nunique().reset_index(name='total_customers')

# Juntar os dois DataFrames para comparar novos clientes com o total
acquisition_rate = pd.merge(new_customers_per_month, total_customers_per_month, on='month_year')

# Calcular a taxa de aquisição de novos clientes
acquisition_rate['acquisition_rate'] = (acquisition_rate['new_customers'] / acquisition_rate['total_customers']) * 100

# Exibir o resultado
print(acquisition_rate)
kpis['acquisition_rate'] = acquisition_rate

### Calcular o CAC
# Custo médio para adquirir um novo cliente = (Total de Gastos em Marketing / Número de Novos Clientes).
# Essencial para entender a eficiência das campanhas de marketing.
# Considerando um valor de R$10.000,00 por mês em campanhas de marketing.

# Gasto mensal em marketing
marketing_spend_per_month = 10000

# Já temos o DataFrame 'new_customers_per_month' com a contagem de novos clientes por mês

# Calcular o CAC para cada mês
new_customers_per_month['CAC'] = marketing_spend_per_month / new_customers_per_month['new_customers']

# Exibir o resultado
print(new_customers_per_month[['month_year', 'new_customers', 'CAC']])
kpis['cac_month'] = new_customers_per_month[['month_year', 'CAC']]

# Converter 'month_year' para datetime para facilitar a extração do ano
new_customers_per_month['year'] = new_customers_per_month['month_year'].dt.year

# Calcular o número total de novos clientes por ano
new_customers_per_year = new_customers_per_month.groupby('year')['new_customers'].sum().reset_index()

# Calcular o gasto total de marketing por ano (gasto mensal multiplicado por 12)
new_customers_per_year['total_marketing_spend'] = marketing_spend_per_month * 12

# Calcular o CAC anual: total gasto em marketing dividido pelo número total de novos clientes por ano
new_customers_per_year['CAC'] = new_customers_per_year['total_marketing_spend'] / new_customers_per_year['new_customers']

# Exibir o resultado
print(new_customers_per_year[['year', 'new_customers', 'total_marketing_spend', 'CAC']])

# Adicionar ao dicionário de KPIs
kpis['cac_year'] = new_customers_per_month[['month_year', 'CAC']]

### Calcular ciclo de vendas
### Calcular nível de estoque
### Calcular vendas por canal
### Enviar os KPIs para o próximo step

orchest.output(kpis, name="kpis")
print(kpis.keys())